# UAS Project Streamlit: 
- **Nama:** [Bahteramon Bintang Sanjaya Manurung]
- **Dataset:** [Dataset Gambar-Gambar Bunga Dalam 5 Jenis meliput Daisy,Dandelion,Roses,Sunflower,Tulip. ("https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz")]
- **URL Website:** [https://bahteramonbintang.streamlit.app/]



## Menentukan Pertanyaan Bisnis

- Berapa akurasi model klasifikasi bunga?
- Apakah model dapat secara efektif mengklasifikasikan berbagai jenis bunga (Margarita, Bunga Matahari, Tulip, Bunga Dandelion, Mawar) berdasarkan gambar?

## Import Semua Packages/Library yang Digunakan

In [ ]:
import os
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import tensorflow as tf
import random as rn
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image

## Data Wrangling

### Gathering Data

In [ ]:
directory_path = r'C:\Users\monba\Desktop\Kuliah III RPLK\Pemrograman Pararel\flower_photos'

print(os.listdir(directory_path))

X=[]
Z=[]
IMG_SIZE=150
FLOWER_DAISY_DIR = r'C:\Users\monba\Desktop\Kuliah III RPLK\Pemrograman Pararel\flower_photos\daisy'
FLOWER_SUNFLOWER_DIR = r'C:\Users\monba\Desktop\Kuliah III RPLK\Pemrograman Pararel\flower_photos\sunflowers'
FLOWER_TULIP_DIR = r'C:\Users\monba\Desktop\Kuliah III RPLK\Pemrograman Pararel\flower_photos\tulips'
FLOWER_DANDI_DIR = r'C:\Users\monba\Desktop\Kuliah III RPLK\Pemrograman Pararel\flower_photos\dandelion'
FLOWER_ROSE_DIR = r'C:\Users\monba\Desktop\Kuliah III RPLK\Pemrograman Pararel\flower_photos\roses'

def assign_label(img,flower_type):
    return flower_type

def make_train_data(flower_type,DIR):
    for img in tqdm(os.listdir(DIR)):
        label=assign_label(img,flower_type)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        
        X.append(np.array(img))
        Z.append(str(label))

### Assessing Data

In [ ]:
make_train_data('Daisy',FLOWER_DAISY_DIR)
print(len(X))
make_train_data('Sunflower',FLOWER_SUNFLOWER_DIR)
print(len(X))
make_train_data('Tulip',FLOWER_TULIP_DIR)
print(len(X))
make_train_data('Dandelion',FLOWER_DANDI_DIR)
print(len(X))
make_train_data('Rose',FLOWER_ROSE_DIR)
print(len(X))

### Cleaning Data

In [ ]:
None

## Exploratory Data Analysis (EDA)

### Explore ...

In [ ]:
le=LabelEncoder()
Y=le.fit_transform(Z)
Y=to_categorical(Y,5)
X=np.array(X)
X=X/255

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.25,random_state=42)

In [ ]:
np.random.seed(42)
rn.seed(42)
tf.random.set_seed(42)

## Visualization & Explanatory Analysis

In [ ]:
fig,ax=plt.subplots(5,2)
fig.set_size_inches(15,15)
for i in range(5):
    for j in range (2):
        l=rn.randint(0,len(Z))
        ax[i,j].imshow(X[l])
        ax[i,j].set_title('Flower: '+Z[l])
        
plt.tight_layout()

## Membuat Model 

In [ ]:

model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
 

model.add(Conv2D(filters =96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters = 96, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(5, activation = "softmax"))

In [ ]:
batch_size=128
epochs=50

from keras.callbacks import ReduceLROnPlateau
red_lr= ReduceLROnPlateau(monitor='val_acc',patience=3,verbose=1,factor=0.1)

In [ ]:
datagen = ImageDataGenerator(
        featurewise_center=False, 
        samplewise_center=False,  
        featurewise_std_normalization=False, 
        samplewise_std_normalization=False,  
        zca_whitening=False,  
        rotation_range=10,  
        zoom_range = 0.1,
        width_shift_range=0.2, 
        height_shift_range=0.2,  
        horizontal_flip=True,  
        vertical_flip=False)  


datagen.fit(x_train)

In [ ]:
model.compile(optimizer=Adam(lr=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.summary()

### Training Model

In [ ]:
History = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size),
                              epochs = epochs, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)

### Evaluasi Model

In [ ]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
plt.plot(History.history['accuracy']) 
plt.plot(History.history['val_accuracy'])  
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
pred=model.predict(x_test)
pred_digits=np.argmax(pred,axis=1)

In [ ]:
i=0
prop_class=[]
mis_class=[]

for i in range(len(y_test)):
    if(np.argmax(y_test[i])==pred_digits[i]):
        prop_class.append(i)
    if(len(prop_class)==8):
        break

i=0
for i in range(len(y_test)):
    if(not np.argmax(y_test[i])==pred_digits[i]):
        mis_class.append(i)
    if(len(mis_class)==8):
        break

In [ ]:
count = 0
fig, ax = plt.subplots(4, 2)
fig.set_size_inches(15, 15)

with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    
    for i in range(4):
        for j in range(2):
            ax[i, j].imshow(x_test[prop_class[count]])
            
            y_test_index = np.argmax(y_test[prop_class[count]])
            
            ax[i, j].set_title("Predicted Flower : " + str(le.inverse_transform([pred_digits[prop_class[count]]])) + "\n" +
                               "Actual Flower : " + str(le.inverse_transform([y_test_index])))
            plt.tight_layout()
            count += 1

In [ ]:
count = 0
fig, ax = plt.subplots(4, 2)
fig.set_size_inches(15, 15)

for i in range(4):
    for j in range(2):
        ax[i, j].imshow(x_test[mis_class[count]])
        
        print("Shape of y_test[mis_class[{}]]: {}".format(count, y_test[mis_class[count]].shape))
        
        y_test_flattened = np.argmax(y_test[mis_class[count]], axis=0)
        
        ax[i, j].set_title("Predicted Flower : " + str(le.inverse_transform([pred_digits[mis_class[count]]])) + "\n" +
                           "Actual Flower : " + str(le.inverse_transform([y_test_flattened])))
        plt.tight_layout()
        count += 1

### Menyimpan Model

In [ ]:
model.save('flower_model.h5')

## Conclusion

- Akurasi model klasifikasi bunga dapat dievaluasi menggunakan metrik yang sesuai, menunjukkan efektivitasnya dalam mengklasifikasikan berbagai jenis bunga.
- Model dapat berhasil mengklasifikasikan gambar bunga ke dalam kategori-kategori mereka dengan tingkat akurasi tertentu, memungkinkan pengguna situs web untuk mengidentifikasi berbagai jenis bunga.